In [1]:
import pandas as pd
import numpy as np
from scipy import stats as stats
import matplotlib.pyplot as plt
import seaborn as sb
import KeyFunctions as me
import tensorflow as tf
RandState = 117

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

n = 4

In [2]:
from sklearn.model_selection import train_test_split

#Import Full Triclosan Dataset
df, labels = me.ConstructCombinedTriclosanDataset()
df = me.BaselineCorrection(me.BaselineCorrection(df))

[train, test] = train_test_split(df, random_state = RandState, shuffle = True, train_size = 0.8)

y_tn = train.index
y_tt = test.index
X_tt = test.to_numpy()
X_tn = train.to_numpy()

#Augment Data to 4000 Spectra
X_tnAu, y_tnAu = me.AugmentData(X_tn, y_tn, 4000, df.columns.to_numpy(), False)

In [3]:
df.index.value_counts()

10-5    11
10-3    10
10-4    10
10-9     9
10-7     6
10-8     6
Name: index, dtype: int64

In [4]:
#Set Training Parameters
verbose = 1
epochsvec = [5, 20, 50]
batch_sizevec = [10, 50, 100]
epochs = epochsvec[1]
batch_size = batch_sizevec[1]

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#Scale X-Data with Training Xs
scaler = StandardScaler()
scaler.fit(X_tnAu)
X_tnS = scaler.transform(X_tnAu)
X_ttS = scaler.transform(X_tt)

#Encode y-Data with Training ys
encoder = LabelEncoder()
encoder.fit(y_tnAu)
y_tn_e = encoder.transform(y_tnAu)
y_tn_p = np_utils.to_categorical(y_tn_e, num_classes = len(labels))
y_tt_e = encoder.transform(y_tt)
y_tt_p = np_utils.to_categorical(y_tt_e, num_classes = len(labels))


#Reshape All Data to a 3D Tensor of Shape [Number of Spectra, Number of Timesteps(1), Number of Wavelengths]
X_tn_p = X_tnS.reshape(X_tnS.shape[0], X_tnS.shape[1], 1)
X_tt_p = X_ttS.reshape(X_ttS.shape[0], X_ttS.shape[1], 1)

y_tnT = tf.convert_to_tensor(y_tn_p)
y_ttT = tf.convert_to_tensor(y_tt_p)
X_tnT = tf.convert_to_tensor(X_tn_p)
X_ttT = tf.convert_to_tensor(X_tt_p)

display(X_tnT.shape)
display(y_tnT.shape)
display(X_ttT.shape)
display(y_ttT.shape)

ytruth = tf.argmax(input = y_ttT, axis = 1).numpy()
ytruth = encoder.inverse_transform(ytruth)

TensorShape([4000, 901, 1])

TensorShape([4000, 6])

TensorShape([11, 901, 1])

TensorShape([11, 6])

In [6]:
#Multi-class Classification with Keras
 
n_timesteps, n_features, n_outputs = X_tn_p.shape[1], X_tn_p.shape[2], y_tn_p.shape[1]

#Define Sequential Model - 1 Convolution Layer, 1 Dropout Layer, 1 Flatten Layer, 2 Dense Layers
model = Sequential()
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Implement EarlyStopping
stopper = tf.keras.callbacks.EarlyStopping(monitor = "val_loss",patience = 2,verbose = 0, restore_best_weights = True)

#Fit Model
model.fit(X_tnT, y_tnT, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_split = 0.2, callbacks = stopper)

#Evaluate Model
_, SCaccuracy = model.evaluate(X_ttT, y_ttT, batch_size=batch_size, verbose=verbose)
display(SCaccuracy)

Epoch 1/20
64/64 [==============================] - 7s 107ms/step - loss: 0.1018 - accuracy: 0.9650 - val_loss: 2.5341e-04 - val_accuracy: 1.0000
Epoch 2/20
64/64 [==============================] - 6s 89ms/step - loss: 1.7429e-05 - accuracy: 1.0000 - val_loss: 1.5824e-04 - val_accuracy: 1.0000
Epoch 3/20
64/64 [==============================] - 6s 97ms/step - loss: 1.1421e-05 - accuracy: 1.0000 - val_loss: 1.3248e-04 - val_accuracy: 1.0000
Epoch 4/20
64/64 [==============================] - 6s 97ms/step - loss: 9.2965e-06 - accuracy: 1.0000 - val_loss: 1.1784e-04 - val_accuracy: 1.0000
Epoch 5/20
64/64 [==============================] - 6s 88ms/step - loss: 8.2282e-06 - accuracy: 1.0000 - val_loss: 1.0682e-04 - val_accuracy: 1.0000
Epoch 6/20
64/64 [==============================] - 7s 108ms/step - loss: 7.4829e-06 - accuracy: 1.0000 - val_loss: 9.7113e-05 - val_accuracy: 1.0000
Epoch 7/20
64/64 [==============================] - 6s 88ms/step - loss: 6.2773e-06 - accuracy: 1.0000 - val

0.5454545617103577

In [7]:
#Make Prediction
SCypred = model.predict(X_ttT)
SCypred = tf.argmax(input = SCypred, axis = 1).numpy()
SCypred = encoder.inverse_transform(SCypred)


In [8]:
#Extract Convolution Feature Maps
convlayer = tf.keras.Model(inputs=model.inputs, outputs=model.layers[0].output)

spectra = X_ttT[n, :, 0]
specvec = list()
specvec.append(spectra)
spectra = tf.reshape(spectra, (1, len(spectra), 1))

SCfeature_maps = convlayer.predict(spectra)
display(SCfeature_maps)

array([[[0.        , 0.260674  , 0.06863199, ..., 0.05131504,
         0.22697566, 0.        ],
        [0.        , 0.25130126, 0.04998049, ..., 0.16755699,
         0.17196947, 0.        ],
        [0.08547481, 0.02918387, 0.03943924, ..., 0.09827796,
         0.        , 0.        ],
        ...,
        [0.0259135 , 0.14349277, 0.03860227, ..., 0.1464171 ,
         0.06791373, 0.        ],
        [0.0584053 , 0.        , 0.00368386, ..., 0.07261916,
         0.        , 0.        ],
        [0.00071223, 0.        , 0.        , ..., 0.00055261,
         0.        , 0.06282327]]], dtype=float32)

In [9]:
#Apply Fourier Transform to Training and Testing Data
X_tnf = np.apply_along_axis(np.fft.fft, axis=1, arr=X_tnAu)
X_ttf = np.apply_along_axis(np.fft.fft, axis=1, arr=X_tt)

#Combine Real and Imaginary Part of FT in form [real, imaginary]
X_tnf = np.append(X_tnf.real, X_tnf.imag, axis = 1)
X_ttf = np.append(X_ttf.real, X_ttf.imag, axis = 1)
X_tnf= X_tnf.astype('float32')
X_ttf= X_ttf.astype('float32')

#Scale X-Data with Training Xs
scaler = StandardScaler()
scaler.fit(X_tnf)
X_tnf = scaler.transform(X_tnf)
X_ttf = scaler.transform(X_ttf)

#Encode y-Data with Training ys
encoder = LabelEncoder()
encoder.fit(y_tnAu)
y_tn_e = encoder.transform(y_tnAu)
y_tn_p = np_utils.to_categorical(y_tn_e, num_classes = len(labels))
y_tt_e = encoder.transform(y_tt)
y_tt_p = np_utils.to_categorical(y_tt_e, num_classes = len(labels))

#Reshape All Data to a 3D Tensor of Shape [Number of Spectra, Number of Timesteps(1), Number of Wavelengths]
X_tn_p = X_tnf.reshape(X_tnf.shape[0], X_tnf.shape[1], 1)
X_tt_p = X_ttf.reshape(X_ttf.shape[0], X_ttf.shape[1], 1)

y_tnT = tf.convert_to_tensor(y_tn_p)
y_ttT = tf.convert_to_tensor(y_tt_p)
X_tnT = tf.convert_to_tensor(X_tn_p)
X_ttT = tf.convert_to_tensor(X_tt_p)


display(X_tnT.shape)
display(y_tnT.shape)
display(X_ttT.shape)
display(y_ttT.shape)

TensorShape([4000, 1802, 1])

TensorShape([4000, 6])

TensorShape([11, 1802, 1])

TensorShape([11, 6])

In [10]:
#Multi-class Classification with Keras
 
n_timesteps, n_features, n_outputs = X_tn_p.shape[1], X_tn_p.shape[2], y_tn_p.shape[1]

#Define Sequential Model - 1 Convolution Layer, 1 Dropout Layer, 1 Flatten Layer, 2 Dense Layers
model = Sequential()
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Implement EarlyStopping
stopper = tf.keras.callbacks.EarlyStopping(monitor = "val_loss",patience = 2,verbose = 0, restore_best_weights = True)

#Fit Model
model.fit(X_tnT, y_tnT, epochs=epochs, batch_size=batch_size, verbose=verbose,  validation_split=0.2, callbacks = stopper)

#Evaluate Model
_, FTaccuracy = model.evaluate(X_ttT, y_ttT, batch_size=batch_size, verbose=verbose)
display(FTaccuracy)

Epoch 1/20
64/64 [==============================] - 13s 196ms/step - loss: 0.2733 - accuracy: 0.9259 - val_loss: 6.8607e-04 - val_accuracy: 1.0000
Epoch 2/20
64/64 [==============================] - 13s 200ms/step - loss: 1.2972e-04 - accuracy: 1.0000 - val_loss: 4.1325e-04 - val_accuracy: 1.0000
Epoch 3/20
64/64 [==============================] - 12s 183ms/step - loss: 8.6668e-05 - accuracy: 1.0000 - val_loss: 3.2247e-04 - val_accuracy: 1.0000
Epoch 4/20
64/64 [==============================] - 12s 183ms/step - loss: 6.5533e-05 - accuracy: 1.0000 - val_loss: 2.5682e-04 - val_accuracy: 1.0000
Epoch 5/20
64/64 [==============================] - 13s 200ms/step - loss: 5.0408e-05 - accuracy: 1.0000 - val_loss: 2.1152e-04 - val_accuracy: 1.0000
Epoch 6/20
64/64 [==============================] - 14s 220ms/step - loss: 4.0549e-05 - accuracy: 1.0000 - val_loss: 1.7675e-04 - val_accuracy: 1.0000
Epoch 7/20
64/64 [==============================] - 13s 196ms/step - loss: 3.2560e-05 - accuracy: 

0.6363636255264282

In [11]:
#Make Prediction
FTypred = model.predict(X_ttT)
FTypred = tf.argmax(input = FTypred, axis = 1).numpy()
FTypred = encoder.inverse_transform(FTypred)

In [12]:
convlayer = tf.keras.Model(inputs=model.inputs, outputs=model.layers[0].output)
convlayer.summary()

spectra = X_ttT[n, :, 0]
specvec.append(spectra)
spectra = tf.reshape(spectra, (1, len(spectra), 1))

FTfeature_maps = convlayer.predict(spectra)
display(FTfeature_maps)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2_input (InputLayer)  [(None, 1802, 1)]         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1800, 64)          256       
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


array([[[0.00422935, 0.10867103, 0.        , ..., 0.01488325,
         0.        , 0.00630847],
        [0.        , 0.16611724, 0.        , ..., 0.04071415,
         0.04526045, 0.07095192],
        [0.        , 0.14044341, 0.        , ..., 0.        ,
         0.13208564, 0.06784447],
        ...,
        [0.        , 0.00634472, 0.04545243, ..., 0.        ,
         0.10038912, 0.01804178],
        [0.        , 0.        , 0.07181382, ..., 0.01811103,
         0.04018268, 0.00832233],
        [0.02018333, 0.00538357, 0.01623065, ..., 0.        ,
         0.        , 0.        ]]], dtype=float32)

In [13]:
#Apply Welsh-Hadamard Transform to Training and Testing Data
from sympy.discrete.transforms import fwht, ifwht


#Scale X-Data with Training Xs
scaler = StandardScaler()
scaler.fit(X_tnAu)
X_tnS = scaler.transform(X_tnAu)
X_ttS = scaler.transform(X_tt)


X_tnh = np.apply_along_axis(fwht, axis=1, arr=X_tnS)
X_tth = np.apply_along_axis(fwht, axis=1, arr=X_ttS)
X_tnh = X_tnh.astype('float32')
X_tth = X_tth.astype('float32')

#Encode y-Data with Training ys
encoder = LabelEncoder()
encoder.fit(y_tnAu)
y_tn_e = encoder.transform(y_tnAu)
y_tn_p = np_utils.to_categorical(y_tn_e, num_classes = len(labels))
y_tt_e = encoder.transform(y_tt)
y_tt_p = np_utils.to_categorical(y_tt_e, num_classes = len(labels))

#Reshape All Data to a 3D Tensor of Shape [Number of Spectra, Number of Timesteps(1), Number of Wavelengths]
X_tn_p = X_tnh.reshape(X_tnh.shape[0], X_tnh.shape[1], 1)
X_tt_p = X_tth.reshape(X_tth.shape[0], X_tth.shape[1], 1)

y_tnT = tf.convert_to_tensor(y_tn_p)
y_ttT = tf.convert_to_tensor(y_tt_p)
X_tnT = tf.convert_to_tensor(X_tn_p)
X_ttT = tf.convert_to_tensor(X_tt_p)


display(X_tnT.shape)
display(y_tnT.shape)
display(X_ttT.shape)
display(y_ttT.shape)

TensorShape([4000, 1024, 1])

TensorShape([4000, 6])

TensorShape([11, 1024, 1])

TensorShape([11, 6])

In [ ]:
#Multi-class Classification with Keras
 
n_timesteps, n_features, n_outputs = X_tn_p.shape[1], X_tn_p.shape[2], y_tn_p.shape[1]

#Define Sequential Model - 1 Convolution Layer, 1 Dropout Layer, 1 Flatten Layer, 2 Dense Layers
model = Sequential()
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Implement EarlyStopping
stopper = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", mode = 'min',\
                                           patience = 2, verbose = 1, restore_best_weights = True)

#Fit Model
model.fit(X_tnT, y_tnT, epochs=epochs, batch_size=batch_size, verbose=verbose,  validation_split=0.2, callbacks = stopper)

#Evaluate Model
_, HTaccuracy = model.evaluate(X_ttT, y_ttT, batch_size=batch_size, verbose=verbose)
display(HTaccuracy)

Epoch 1/20
64/64 [==============================] - 7s 106ms/step - loss: 2.8483 - accuracy: 0.8850 - val_loss: 9.5927e-05 - val_accuracy: 1.0000
Epoch 2/20
64/64 [==============================] - 8s 119ms/step - loss: 3.0202e-05 - accuracy: 1.0000 - val_loss: 2.2254e-05 - val_accuracy: 1.0000
Epoch 3/20
64/64 [==============================] - 7s 103ms/step - loss: 9.2763e-06 - accuracy: 1.0000 - val_loss: 1.0594e-05 - val_accuracy: 1.0000
Epoch 4/20
64/64 [==============================] - 8s 120ms/step - loss: 9.0701e-06 - accuracy: 1.0000 - val_loss: 9.8067e-06 - val_accuracy: 1.0000
Epoch 5/20
64/64 [==============================] - 7s 102ms/step - loss: 4.5109e-06 - accuracy: 1.0000 - val_loss: 6.3963e-06 - val_accuracy: 1.0000
Epoch 6/20
64/64 [==============================] - 7s 112ms/step - loss: 3.1261e-06 - accuracy: 1.0000 - val_loss: 4.6644e-06 - val_accuracy: 1.0000
Epoch 7/20
64/64 [==============================] - 7s 108ms/step - loss: 2.4036e-06 - accuracy: 1.0000 

In [ ]:
#Make Prediction
HTypred = model.predict(X_ttT)
HTypred = tf.argmax(input = HTypred, axis = 1).numpy()
HTypred = encoder.inverse_transform(HTypred)

In [ ]:
convlayer = tf.keras.Model(inputs=model.inputs, outputs=model.layers[0].output)
convlayer.summary()

spectra = X_ttT[n, :, 0]
specvec.append(spectra)
spectra = tf.reshape(spectra, (1, len(spectra), 1))

HTfeature_maps = convlayer.predict(spectra)
display(HTfeature_maps)

In [ ]:
from sklearn.metrics import confusion_matrix
display(SCypred)
CMSC = confusion_matrix(ytruth, SCypred, labels = labels)
CMFT = confusion_matrix(ytruth, FTypred, labels = labels)
CMHT = confusion_matrix(ytruth, HTypred, labels = labels)
display(SCaccuracy, FTaccuracy, HTaccuracy)

In [ ]:
fig, axs = plt.subplots(1, 3, sharey = True, figsize = [15, 5])

plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True

pcm = axs[0].pcolor(CMSC, edgecolors = 'k', cmap = 'gist_heat_r')
plt.gca().invert_yaxis()
axs[0].set_xticks(ticks = np.linspace(0.5, len(labels)-0.5, num = len(labels)), labels = labels)
axs[0].set_yticks(ticks = np.linspace(0.5, len(labels)-0.5, num = len(labels)), labels = labels)
axs[0].set_ylabel("Actual Condition")
axs[0].set_xlabel("Predicted Condition")
axs[0].xaxis.set_label_position('top') 
axs[0].set_title('Raw Data');

axs[1].pcolor(CMFT, edgecolors = 'k', cmap = 'gist_heat_r');
plt.gca().invert_yaxis()
axs[1].set_xticks(ticks = np.linspace(0.5, len(labels)-0.5, num = len(labels)), labels = labels)
axs[1].set_title('Fourier Transform');
axs[1].set_xlabel("Predicted Condition")
axs[1].xaxis.set_label_position('top')

axs[2].pcolor(CMHT, edgecolors = 'k', cmap = 'gist_heat_r')
plt.gca().invert_yaxis()
axs[2].set_xticks(ticks = np.linspace(0.5, len(labels)-0.5, num = len(labels)), labels = labels);
axs[2].set_title('Walsh Hadamard Transform');
axs[2].set_xlabel("Predicted Condition")
axs[2].xaxis.set_label_position('top')

fig.colorbar(pcm, ax = axs[:], location = 'bottom', label = 'Number of Assignments');